In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch_geometric.datasets import TUDataset
from torch_geometric.utils.convert import to_networkx
from toponetx.classes.simplicial_complex import SimplicialComplex

from topomodelx.nn.hypergraph.unigcn import UniGCN
import warnings

warnings.filterwarnings("ignore")

# Train a UNIGCN TNN

# Pre-processing

## Import data ##

The first step is to import the dataset, MUTAG, a benchmark dataset for graph classification. We then lift each graph into our domain of choice, a hypergraph.

We will also retrieve:
- input signal on the edges for each of these hypergraphs, as that will be what we feed the model in input
- the binary label associated to the hypergraph

In [2]:
dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG", use_edge_attr=True)
dataset = dataset[:100]
hg_list = []
x_1_list = []
y_list = []
for graph in dataset:
    hg = SimplicialComplex(to_networkx(graph)).to_hypergraph()
    hg_list.append(hg)
    x_1_list.append(graph.x)
    y_list.append(int(graph.y))

incidence_1_list = []
for hg in hg_list:
    incidence_1 = hg.incidence_matrix()
    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse_csr()
    incidence_1_list.append(incidence_1)

# Create the Neural Network

Using the UniGCNLayer class, we create a neural network with stacked layers.

In [3]:
channels_edge = x_1_list[0].shape[1]
channels_node = dataset[0].x.shape[1]
model = UniGCN(channels_edge, channels_node, n_layers=2)

# Train the Neural Network

We specify the model, the loss, and an optimizer.

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
crit = torch.nn.BCELoss()

Split the dataset into train, val, and test sets.

In [5]:
x_1_train, x_1_test = train_test_split(x_1_list, test_size=0.2, shuffle=False)
incidence_1_train, incidence_1_test = train_test_split(
    incidence_1_list, test_size=0.2, shuffle=False
)
y_train, y_test = train_test_split(y_list, test_size=0.2, shuffle=False)

x_1_train, x_1_val = train_test_split(x_1_train, test_size=0.2, shuffle=False)
incidence_1_train, incidence_1_val = train_test_split(
    incidence_1_train, test_size=0.2, shuffle=False
)
y_train, y_val = train_test_split(y_train, test_size=0.2, shuffle=False)

The following cell performs the training, looping over the network for a low amount of epochs. 

**Note: The number of epochs below have been kept low to facilitate debugging and testing. Real use cases should likely require more epochs.**

In [6]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    loss = 0
    for x_1, incidence_1, y in zip(x_1_train, incidence_1_train, y_train):
        output = model(x_1.float(), incidence_1.float())
        loss += crit(output, torch.tensor([y]).float())
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch} loss: {loss.item()}")
    model.eval()
    with torch.no_grad():
        correct = 0
        for x_1, incidence_1, y in zip(x_1_val, incidence_1_val, y_val):
            output = model(x_1.float(), incidence_1.float())
            pred = output > 0.5
            if pred == y:
                correct += 1
        print(f"Epoch {epoch} Validation accuracy: {correct / len(y_val)}")

model.eval()
with torch.no_grad():
    correct = 0
    for x_1, incidence_1, y in zip(x_1_test, incidence_1_test, y_test):
        output = model(x_1.float(), incidence_1.float())
        pred = output > 0.5
        if pred == y:
            correct += 1
    print(f"Test accuracy: {correct / len(y_test)}")

Epoch 0 loss: 902.4901733398438
Epoch 0 Validation accuracy: 0.4375
Epoch 1 loss: 742.79052734375
Epoch 1 Validation accuracy: 0.4375
Epoch 2 loss: 585.759521484375
Epoch 2 Validation accuracy: 0.4375
Epoch 3 loss: 432.1170654296875
Epoch 3 Validation accuracy: 0.4375
Epoch 4 loss: 282.8601379394531
Epoch 4 Validation accuracy: 0.4375
Epoch 5 loss: 143.66136169433594
Epoch 5 Validation accuracy: 0.75
Epoch 6 loss: 55.54958724975586
Epoch 6 Validation accuracy: 0.5625
Epoch 7 loss: 50.33615493774414
Epoch 7 Validation accuracy: 0.5625
Epoch 8 loss: 72.13643646240234
Epoch 8 Validation accuracy: 0.5625
Epoch 9 loss: 92.25325775146484
Epoch 9 Validation accuracy: 0.5625
Epoch 10 loss: 108.162841796875
Epoch 10 Validation accuracy: 0.5625
Epoch 11 loss: 119.53228759765625
Epoch 11 Validation accuracy: 0.5625
Epoch 12 loss: 126.69290924072266
Epoch 12 Validation accuracy: 0.5625
Epoch 13 loss: 129.98519897460938
Epoch 13 Validation accuracy: 0.5625
Epoch 14 loss: 130.16600036621094
Epoch 14